In [1]:
import xarray as xr
import glob
import os
import numpy as np
import pandas as pd
from dask.diagnostics import ProgressBar
import time
from dask import delayed
from dask import delayed, persist
import dask
import xarray as xr
from dask import delayed
from dask.diagnostics import ProgressBar


In [2]:
#### settings !!! MODIFY THIS BLOCK
start_date = '2012-01-01'
end_date = '2012-12-31' #make sure this date is after the start date... 
interval_hours = 1 #what hour interval would you like to get? [i.e: 1 = 24 files/day, 6 = 4 files/day]
FPout = '/glade/derecho/scratch/wchapman/ERA5_regrid_out/' #where do you want the files stored?
prefix_out = 'ERA5_e5.oper.ml.v3' #what prefix do you want the files stored with?
project_num = 'NAML0001'
#### settings !!! MODIFY THIS BLOCK

if 'client' in locals():
    client.shutdown()
    print('...shutdown client...')
else:
    print('client does not exist yet')
    
###dask NCAR client: 
from distributed import Client
from dask_jobqueue import PBSCluster

cluster = PBSCluster(project='NAML0001',walltime='12:00:00',cores=1, memory='25GB',shared_temp_directory='/glade/scratch/wchapman/tmp',queue='casper')
cluster.scale(jobs=40)
client = Client(cluster)
client

client does not exist yet


/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)
/glade/u/apps/opt/conda/envs/npl-2023b/lib/python3.10/site-packages/dask_jobqueue/pbs.py:82: FutureWarning: project has been renamed to account as this kwarg was used wit -A option. You are still using it (please also check config files). If you did not set account yet, project will be respected for now, but it will be removed in a future release. If you already set account, project is ignored and you can remove it.
  warnings.warn(warn, FutureWarning)


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dingo/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dingo/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.67:45655,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/wchapman/Dingo/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [3]:
def find_strings_with_substring(string_list, substring):
    # Initialize an empty list to store matching strings
    matching_strings = []

    # Iterate through the list
    for string in string_list:
        # Check if the specified substring is present in the current string
        if substring in string:
            matching_strings.append(string)

    # Return the list of matching strings
    return matching_strings

def flatten_list(input_list):
    flattened_list = []
    for item in input_list:
        if isinstance(item, list):
            flattened_list.extend(flatten_list(item))
        else:
            flattened_list.append(item)
    return flattened_list

##function get file paths ... 
def fp_dates_wanted(Dateswanted):
    years_wanted = Dateswanted[:].year
    months_wanted = Dateswanted[:].month
    day_wanted = Dateswanted[:].day
    
    list_yrm =[]
    for ywmw in zip(years_wanted,months_wanted):
        list_yrm.append(str(ywmw[0])+f'{ywmw[1]:02}')
    
    fp_t = []
    fp_u = []
    fp_v = []
    fp_q = []
    fp_ps = []
    
    lastday = str(Dateswanted[-1])[:10]
    
    for yrm_fp in np.unique(list_yrm):
        for dayday in np.unique(day_wanted):
            
            
            fp_u.append(sorted(glob.glob('/glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml/'+yrm_fp+'/'+'*_u*'+yrm_fp+f'{dayday:02}'+'*.nc')))
            fp_v.append(sorted(glob.glob('/glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml/'+yrm_fp+'/'+'*_v*'+yrm_fp+f'{dayday:02}'+'*.nc')))
            fp_t.append(sorted(glob.glob('/glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml/'+yrm_fp+'/'+'*_t*'+yrm_fp+f'{dayday:02}'+'*.nc')))
            fp_q.append(sorted(glob.glob('/glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml/'+yrm_fp+'/'+'*_q*'+yrm_fp+f'{dayday:02}'+'*.nc')))
            fp_ps.append(sorted(glob.glob('/glade/campaign/collections/rda/data/ds633.6/e5.oper.an.ml/'+yrm_fp+'/'+'*_sp*'+yrm_fp+f'{dayday:02}'+'*.nc')))
            
            if yrm_fp[:4]+'-'+yrm_fp[4:]+'-'+f'{dayday:02}' == lastday:
                break

    fp_u = flatten_list(fp_u)
    fp_v = flatten_list(fp_v)
    fp_t = flatten_list(fp_t)
    fp_q = flatten_list(fp_q)
    fp_ps = flatten_list(fp_ps)
    
    files_dict ={'u':np.unique(fp_u),'v':np.unique(fp_v),'t':np.unique(fp_t),'q':np.unique(fp_q),'ps':np.unique(fp_ps)}
    
    
    return files_dict 

def make_nc_files(files_dict,Dateswanted,Dayswanted):    
    for dw in Dayswanted:
        print(str(dw)[:10])
        substring_match = str(dw)[:4]+str(dw)[5:7]+str(dw)[8:10]
        smatch_u = find_strings_with_substring(files_dict['u'], substring_match)
        smatch_v = find_strings_with_substring(files_dict['v'], substring_match)
        smatch_t = find_strings_with_substring(files_dict['t'], substring_match)
        smatch_q = find_strings_with_substring(files_dict['q'], substring_match)
        smatch_ps = find_strings_with_substring(files_dict['ps'], substring_match)
        DS_u= xr.open_mfdataset(smatch_u)
        sel_times = Dateswanted.intersection(DS_u['time'])
        DS_v= xr.open_mfdataset(smatch_v).sel(time=sel_times)
        DS_t= xr.open_mfdataset(smatch_t).sel(time=sel_times)
        DS_q= xr.open_mfdataset(smatch_q).sel(time=sel_times)
        DS_ps= xr.open_mfdataset(smatch_ps).sel(time=sel_times)
        print('loading')
        DS=xr.merge([DS_u.sel(time=sel_times),DS_v,DS_t,DS_q]).load()
        print('loaded')
        
        for ee,tt in enumerate(DS['time']):
            hourdo = DS['time.hour'][ee]
            
            datstr = str(dw)[:4]+str(dw)[5:7]+str(dw)[8:10]+f'{hourdo:02}'
            #DS.sel(time=tt).squeeze().to_netcdf()
            out_file=+'/' +prefix_out +'.uvtq.'+ datstr+'.nc'
            write_job = DS.sel(time=tt).squeeze().to_netcdf(out_file,compute=False)
            with ProgressBar():
                print(f"Writing to {out_file}")
                write_job.compute()      
            print(out_file) 
            out_file=FPout+'/' +prefix_out +'.ps.'+ datstr+'.nc'
            DS_ps['Z_GDS4_SFC'] = xr.zeros_like(DS_ps['SP'])
            DS_ps['Z_GDS4_SFC'][:,:]=Static_zheight['Z_GDS4_SFC'].values
            write_job = DS_ps.sel(time=tt).squeeze().to_netcdf(out_file,compute=False)
            with ProgressBar():
                print(f"Writing to {out_file}")
                write_job.compute()    
            print(out_file) 

    return DS,DS_ps


def make_nc_files_optimized(files_dict, Dateswanted, Dayswanted, FPout, prefix_out):
    """
    Optimized function to perform a specific task using Dask with specified resources.

    Parameters:
    - files_dict: A dictionary of files.
    - Dateswanted: List of dates.
    - Dayswanted: List of days.
    - FPout: Output file path.
    - prefix_out: Output file prefix.

    Returns:
    - delayed_writes: List of delayed write operations.
    """
    Static_zheight = xr.open_dataset('/glade/u/home/wchapman/RegriddERA5_CAMFV/static_operation_ERA5_zhght.nc')
    
    delayed_writes = []
    for dw in Dayswanted:
        print(str(dw)[:10])
        substring_match = str(dw)[:4] + str(dw)[5:7] + str(dw)[8:10]
        smatch_u = find_strings_with_substring(files_dict['u'], substring_match)
        smatch_v = find_strings_with_substring(files_dict['v'], substring_match)
        smatch_t = find_strings_with_substring(files_dict['t'], substring_match)
        smatch_q = find_strings_with_substring(files_dict['q'], substring_match)
        smatch_ps = find_strings_with_substring(files_dict['ps'], substring_match)
        
        DS_u = xr.open_mfdataset(smatch_u, parallel=True)
        sel_times = Dateswanted.intersection(DS_u['time'])
        DS_v = xr.open_mfdataset(smatch_v, parallel=True).sel(time=sel_times)
        DS_t = xr.open_mfdataset(smatch_t, parallel=True).sel(time=sel_times)
        DS_q = xr.open_mfdataset(smatch_q, parallel=True).sel(time=sel_times)
        DS_ps = xr.open_mfdataset(smatch_ps, parallel=True).sel(time=sel_times)
        
        print('loading')
        DS = xr.merge([DS_u.sel(time=sel_times), DS_v, DS_t, DS_q])
        print('copying variables')
        DS['US'] = DS['U'].copy(deep=True)
        DS['VS'] = DS['V'].copy(deep=True)
        print('loaded')
        
        for ee, tt in enumerate(DS['time']):
            hourdo = DS['time.hour'][ee]
            datstr = str(dw)[:4] + str(dw)[5:7] + str(dw)[8:10] + f'{hourdo:02}'
            
            out_file_uvtq = FPout + '/' + prefix_out + '.uvtq.' + datstr + '.nc'
            delayed_write_uvtq = delayed(DS.sel(time=tt).squeeze().to_netcdf)(out_file_uvtq)
            delayed_writes.append(delayed_write_uvtq)
            
            out_file_ps = FPout + '/' + prefix_out + '.ps.' + datstr + '.nc'
            DS_ps['Z_GDS4_SFC'] = xr.zeros_like(DS_ps['SP'])
            DS_ps['Z_GDS4_SFC'][:, :] = Static_zheight['Z_GDS4_SFC'].values
            delayed_write_ps = delayed(DS_ps.sel(time=tt).squeeze().to_netcdf)(out_file_ps)
            delayed_writes.append(delayed_write_ps)

    # Compute the delayed write operations concurrently
    with ProgressBar():
        delayed_writes = list(dask.compute(*delayed_writes))

    return delayed_writes



def divide_datetime_index(date_index, max_items_per_division=4):
    """
    Divide a DatetimeIndex into sublists with a maximum number of items per division.

    Parameters:
    - date_index: DatetimeIndex to be divided.
    - max_items_per_division: Maximum number of items per division (default is 4).

    Returns:
    - divided_lists: List of sublists.
    """
    # Initialize an empty list to store the divided lists
    divided_lists = []

    # Initialize a sublist with the first date
    sublist = [date_index[0]]

    # Iterate through the remaining dates
    for date in date_index[1:]:
        # Add the current date to the sublist
        sublist.append(date)

        # Check if the sublist has reached the maximum allowed size
        if len(sublist) == max_items_per_division:
            # If it has, add the sublist to the divided_lists and reset the sublist
            divided_lists.append(sublist)
            sublist = []

    # If there are remaining items in the sublist, add it to the divided_lists
    if sublist:
        divided_lists.append(sublist)

    # Ensure that every division has at least two items by merging the last two divisions if necessary
    if len(divided_lists[-1]) < 2 and len(divided_lists) > 1:
        last_two_lists = divided_lists[-2:]  # Get the last two divisions
        combined_list = sum(last_two_lists, [])  # Combine them
        divided_lists = divided_lists[:-2]  # Remove the last two divisions
        divided_lists.append(combined_list)  # Add the combined list back

    return divided_lists

def increment_date_by_one_day(date_str):
    """
    Increment a date by one day and return it as a string.

    Parameters:
    - date_str: Input date string in the format 'YYYY-MM-DD'.

    Returns:
    - incremented_date_str: Date string incremented by one day.
    """
    # Convert the input date string to a pandas Timestamp
    date = pd.Timestamp(date_str)

    # Increment the date by one day
    incremented_date = date + pd.DateOffset(days=1)

    # Convert the incremented date back to a string in the same format
    incremented_date_str = incremented_date.strftime('%Y-%m-%d')

    return incremented_date_str

# if __name__ == '__main__':
#     #look at all the dates:
#     Dayswantedtot = pd.date_range(start=start_date,end=end_date,freq=str(interval_hours)+'D')
#     #look at all the dates:
#     print(len(Dayswantedtot))
#     if len(Dayswantedtot)<4:
#         start_time = time.time()  # Record the start time
#         Dayswanted = pd.date_range(start=start_date,end=end_date,freq=str(interval_hours)+'D')
#         Dateswanted = pd.date_range(start=start_date,end=end_date,freq=str(interval_hours)+'H')
#         Static_zheight = xr.open_dataset('/glade/u/home/wchapman/RegriddERA5_CAMFV/static_operation_ERA5_zhght.nc')
#         files_dict=fp_dates_wanted(Dateswanted)
#         #make the files:
#         print('...starting processing...')
#         delayed_writes = make_nc_files_optimized(files_dict, Dateswanted, Dayswanted,FPout, prefix_out)
#         elapsed_time = time.time() - start_time
#         print(f" executed in {elapsed_time} seconds")
#     else: 
#         print('in here!!')
#         divided_lists =divide_datetime_index(Dayswantedtot)

#         for dd in divided_lists:
#             strtd = str(dd[0])[:10]
#             endd  = str(dd[-1])[:10]
#             endd  = increment_date_by_one_day(endd)
#             print('doing files:',strtd,endd)
#             start_time = time.time()  # Record the start time
#             Dayswanted = pd.date_range(start=strtd,end=endd,freq=str(interval_hours)+'D')
#             Dateswanted = pd.date_range(start=strtd,end=endd,freq=str(interval_hours)+'H')
#             Static_zheight = xr.open_dataset('/glade/u/home/wchapman/RegriddERA5_CAMFV/static_operation_ERA5_zhght.nc')
#             files_dict=fp_dates_wanted(Dateswanted)
#             #make the files:
#             print('...starting processing...')
#             delayed_writes = make_nc_files_optimized(files_dict, Dateswanted, Dayswanted,FPout, prefix_out)
#             elapsed_time = time.time() - start_time
#             print(f" phase executed in {elapsed_time} seconds")
            
    
#     if 'client' in locals():
#         client.shutdown()
#         print('...shutdown client...')
#     else:
#         print('client does not exist yet')

In [4]:
start_date = '2012-01-01'
end_date = '2013-01-01' #make sure this date is after the start date... 
interval_hours = 1 #what hour interval would you like to get? [i.e: 1 = 24 files/day, 6 = 4 files/day]
Dayswantedtot = pd.date_range(start=start_date,end=end_date,freq=str(interval_hours)+'D')
Dayswanted = pd.date_range(start=start_date,end=end_date,freq=str(interval_hours)+'D')
Dateswanted = pd.date_range(start=start_date,end=end_date,freq=str(interval_hours)+'H')
Static_zheight = xr.open_dataset('/glade/u/home/wchapman/RegriddERA5_CAMFV/static_operation_ERA5_zhght.nc')
#fp_dates_wanted
print('finding files')
files_dict=fp_dates_wanted(Dateswanted)

finding files


## Hit It

In [137]:
start_date = '2012-01-01'
end_date = '2013-01-01' #make sure this date is after the start date... 
params = {'start_date':start_date,'end_date':end_date,'interval_hours':interval_hours,'levs_train':[10,20,30,40,50,60,70,80,90,100,110,120,130]}
trainset = INC_Dataset_mf(params)

u


In [78]:
# from torch.utils.data import Dataset
import random
# print("PyTorch Version: ", torch.__version__)
def load_files(fnms,**kwargs):
    lvs=kwargs['levs']
    DSm = xr.open_mfdataset(fnms,drop_variables=['weight','utc_date','a_half','zero','a_model','b_model','b_half'],parallel=True).sel(level=lvs)
    return DSm

# class INC_Dataset_mf(Dataset):

class INC_Dataset_mf():
    def __init__(self,params,shuff=True):
        self.params = params
        self.start_date = params['start_date']
        self.end_date = params['end_date']
        self.levels = params['levs_train']
        self.dayswanted = pd.date_range(start=start_date,end=end_date,freq=str(interval_hours)+'D')
        self.dateswanted = pd.date_range(start=start_date,end=end_date,freq=str(interval_hours)+'H')
        self.static_zheight = xr.open_dataset('/glade/u/home/wchapman/RegriddERA5_CAMFV/static_operation_ERA5_zhght.nc')
        self.static_topo = np.expand_dims(np.array(self.static_zheight['Z_GDS4_SFC']),axis=[0])
        self.files_dict = fp_dates_wanted(self.dateswanted)
        first_key = next(iter(self.files_dict))
        self.num_files = len(self.files_dict[first_key])
                        
        self.steps_per_file= len(xr.open_dataset(self.files_dict[first_key][0])['time'])
        self.total_ts = (self.num_files*self.steps_per_file)-self.steps_per_file # this is done so the last file can utilize 2 steps 
        self.file_tracker = list(np.arange(self.num_files-1)) # this is done so the last file can utilize 2 steps 
        self.samples_todo = np.arange(self.steps_per_file) 
        
        self.shuff =shuff
        if self.shuff:
            olist = list(np.arange(self.num_files-1))
            self.file_tracker=list(random.sample(olist, len(olist)))
            self.samples_todo = list(np.arange(self.steps_per_file))
        else:
            self.file_tracker = list(np.arange(self.num_files-1))
            self.samples_todo = list(np.arange(self.steps_per_file))
           
        self.fchoice=0
        self.load_new(self.fchoice,self.params)
        self.count_up=0
        
    def __len__(self):
        return int(self.total_ts)
    
    def load_new(self, file_index, params):

        self.DSm = self.preprocess(file_index, self.params)
        self.x_ = np.array(dedo.DSm.load().to_array())
        
    def preprocess(self,train_data,params):
        
        keys=list(dedo.files_dict)
        
        keys_files = list(dedo.files_dict)
        fls = []
        for kf in keys_files:
            fls.append(dedo.files_dict[kf][file_index])
            fls.append(dedo.files_dict[kf][file_index+1])
        
        
        load_data_kwargs={'levs':self.levels}
        
        train_ = load_files(fls,**load_data_kwargs)
        
        return train_
        
    def __getitem__(self, idx):
        sample_idx = self.get_indices(idx)
                
        # Update sample index for the current file
        # If all samples in the current file are processed, move to the next file
        if self.count_up >= len(.samples_todo):
            self.fchoice+=1
            
            try: 
                self.load_new(self.fchoice, self.params)
            except: 
                self.load_new(0, self.params)
            
            
            if self.shuff:
                self.samples_todo = list(np.arange(self.steps_per_file))             
            else: 
                self.samples_todo = list(np.arange(self.steps_per_file))
            self.count_up = 0
            
        x_pre = self.x_[:,sample_idx,:,:,:]
        x_pre_shp = x_pre.shape
        x = np.reshape(x_pre,[x_pre_shp[0]*x_pre_shp[1],x_pre_shp[2],x_pre_shp[3]])
        x = np.concatenate([x,dedo.static_topo],axis=0)
        
        y1_pre = self.x_[:,sample_idx+1,:,:,:]
        y1 = np.reshape(y1_pre,[x_pre_shp[0]*x_pre_shp[1],x_pre_shp[2],x_pre_shp[3]])
        
        y2_pre = self.x_[:,sample_idx+2,:,:,:]
        y2 = np.reshape(y2_pre,[x_pre_shp[0]*x_pre_shp[1],x_pre_shp[2],x_pre_shp[3]])
        self.count_up += 1
        
        x = torch.tensor(x).float()
        y1 = torch.tensor(y1).float()
        y2 = torch.tensor(y2).float()
        y  = (y1,y2)
        
        return x,y
    
    def get_indices(self, global_idx):
        sample_idx = self.samples_todo[self.count_up]
        return sample_idx

In [130]:
fns[0]
fns[0].replace("train", "val")

'/glade/scratch/wchapman/DA_ML/CESML_AI/Data/daychunks/f.e21.FHIST.f09_f09_mg17_nudge_it_ERA5_19990101_19990110.val.000.nc'

In [134]:
import shutil 
for ee,ff in enumerate(fns[11:14]): 
    new_name = ff.replace("train", "test")
    print(new_name)
    shutil.copy2(ff, new_name)

/glade/scratch/wchapman/DA_ML/CESML_AI/Data/daychunks/f.e21.FHIST.f09_f09_mg17_nudge_it_ERA5_19990421_19990430.test.011.nc
/glade/scratch/wchapman/DA_ML/CESML_AI/Data/daychunks/f.e21.FHIST.f09_f09_mg17_nudge_it_ERA5_19990501_19990510.test.012.nc
/glade/scratch/wchapman/DA_ML/CESML_AI/Data/daychunks/f.e21.FHIST.f09_f09_mg17_nudge_it_ERA5_19990511_19990520.test.013.nc
